In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.dummy import DummyClassifier

In [2]:
dados = pd.read_csv('machine-learning-carros.csv')

In [3]:
dados.drop(['Unnamed: 0'], axis= 1, inplace= True)

In [4]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   preco            10000 non-null  float64
 1   vendido          10000 non-null  int64  
 2   idade_do_modelo  10000 non-null  int64  
 3   km_por_ano       10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [6]:
x= dados.drop(['vendido'], axis= 1)
y= dados['vendido']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .30, stratify= y)

In [7]:
SEED = 133

### **Performance do Modelo com Dados Aleatórios**

In [8]:
np.random.seed = SEED

dummy_stratified = DummyClassifier()
dummy_stratified.fit(x_train, y_train)
acuracia = dummy_stratified.score(x_test, y_test) * 100

print(f"A acurácia do dummy stratified {acuracia:.2f}%")

A acurácia do dummy stratified 58.00%


### **Performance do Modelo com Dados de Teste**

In [9]:

np.random.seed = SEED

random = RandomForestClassifier(criterion= 'gini')
random.fit(x_train, y_train)
previsoes = random.predict(x_test)

acuracia = accuracy_score(y_test, previsoes)

print(f'Acurácia do Modelo {acuracia * 100 :.2f}%') 



Acurácia do Modelo 75.07%


### **Performance do Modelo com dados de Treino**

In [10]:
np.random.seed = SEED

random = RandomForestClassifier(criterion= 'entropy')
random.fit(x_test, y_test)
previsoes = random.predict(x_train)

acuracia = accuracy_score(y_train, previsoes)

print(f'Acurácia do Modelo {acuracia * 100 :.2f}%') 

Acurácia do Modelo 75.97%


### **Performance do Modelo com Validação Cruzada**

In [11]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

np.random.seed = SEED

#Definindo os KFolds
kfold = KFold(n_splits= 10, shuffle= True, random_state= 5)

resultado_validacao_cruzada = cross_val_score(random, x, y, cv = kfold)
resultado = resultado_validacao_cruzada.mean()
desvio = resultado_validacao_cruzada.std()


print(f'Resultado {round((resultado - 2 * desvio) *100)}% e {round((resultado + 2 * desvio) *100)}%')

Resultado 73% e 78%


### **Resultado com StratifiedKFold**

Este geralmente é o método mais indicado quando estamos trabalhando com classes desbalanceadas. 

In [12]:
from sklearn.model_selection import StratifiedKFold

np.random.seed = SEED

cv = StratifiedKFold(n_splits= 10, shuffle= True)

modelo = RandomForestClassifier()

resultado_validacao_cruzada = cross_val_score(modelo, x, y, cv = cv)
resultado = resultado_validacao_cruzada.mean()
desvio = resultado_validacao_cruzada.std()

print(f'Resultado {round((resultado - 2 * desvio) *100)}% e {round((resultado + 2 * desvio) *100)}%')

Resultado 73% e 77%


### **Gerando Modelos de Carros de forma Aleatória**

In [13]:
#np.random.seed(SEED)
dados['modelo']= dados['idade_do_modelo'] + np.random.randint(-2, 3, size= dados.shape[0])
dados['modelo']= dados['modelo'] + abs(dados['modelo'].min()) + 1 
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,22
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,5
4,117384.68,1,4,12770.11290,5


In [14]:
# Verificando os modelos gerados
dados['modelo'].unique()

array([22, 20, 14,  5, 13, 17, 11, 21, 18,  2, 12,  9,  7,  4,  8, 15,  6,
       16, 19, 24, 23, 10,  3,  1])

### **Validação por Grupos**

Prestar atenção ao parametro Groups. É ele quem informa para o validador qual a coluna do grupo.

In [15]:
from sklearn.model_selection import GroupKFold

np.random.seed = SEED

cv = GroupKFold(n_splits= 10)

modelo = RandomForestClassifier()

resultado_validacao_cruzada = cross_val_score(modelo, x, y, cv = cv, groups= dados['modelo'])
resultado = resultado_validacao_cruzada.mean()
desvio = resultado_validacao_cruzada.std()

print(f'Resultado {round((resultado - 2 * desvio) *100)}% e {round((resultado + 2 * desvio) *100)}%')

Resultado 73% e 78%


## **Trabalhando com SVC**

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scalar = StandardScaler()
x_treino_escalado= scalar.fit_transform(x_train)
x_teste_escalado= scalar.fit_transform(x_test)


np.random.seed= SEED

svc = SVC()
svc.fit(x_treino_escalado, y_train)
previsoes= svc.predict(x_teste_escalado)


acuracia = accuracy_score(y_test, previsoes) *100

print(f'Acurácia do Modelo {round(acuracia,2)}%')

Acurácia do Modelo 75.7%


## **SVC de forma errada**

Aqui o correto é utilizar um pipeline porque os dados tem de serem pré-processados.

In [25]:
from sklearn.model_selection import GroupKFold

np.random.seed = SEED

cv = GroupKFold(n_splits= 10)

modelo = SVC()

resultado_validacao_cruzada = cross_val_score(modelo, x, y, cv = cv, groups= dados['modelo'])
resultado = resultado_validacao_cruzada.mean()
desvio = resultado_validacao_cruzada.std()

print(f'Resultado {round((resultado - 2 * desvio) *100)}% e {round((resultado + 2 * desvio) *100)}%')

Resultado 75% e 80%


## **Trabalhando com PipeLine - Forma Correta**

In [28]:
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline

np.random.seed = SEED

cv = GroupKFold(n_splits= 10)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

resultado_validacao_cruzada = cross_val_score(pipeline, x, y, cv = cv, groups= dados['modelo'])
resultado = resultado_validacao_cruzada.mean()
desvio = resultado_validacao_cruzada.std()

print(f'Acurácia média do Modelo {round(resultado * 100,2)}%')
print(f'Resultado {round((resultado - 2 * desvio) *100)}% e {round((resultado + 2 * desvio) *100)}%')

Acurácia média do Modelo 76.6%
Resultado 74% e 79%
